In [ ]:
# Import required translator package 
!pip install deep-translator

In [ ]:
import pandas as pd
from deep_translator import GoogleTranslator

# Load the CSV file
input_file = "CarOwnersChina.csv.csv"  # Replace with your file path
df = pd.read_csv(input_file, low_memory=False)

# Initialize the Google Translator
translator = GoogleTranslator(source='auto', target='en')

# Translate the column headers from Chinese to English
translated_columns = [translator.translate(col) for col in df.columns]

# Replace the column headers with the translated ones
df.columns = translated_columns

# Save the translated DataFrame to a new CSV file
output_file = r"C:\Users\PROTEXXA\Desktop\NationWideChina\CarOwners_translated.csv"  # Replace with your desired output path
df.to_csv(output_file, index=False)

print(f"Translation of column headers complete. Saved to {output_file}")


In [ ]:
import pandas as pd
import os
import sys

# Read the initial CSV file with separator modification and low memory usage
try:
    df = pd.read_csv('CarOwners_translated.csv', low_memory=False)
    if df.empty:
        print("Warning: The input CSV file is empty.")
    else:
        print("CSV file loaded successfully.")
except FileNotFoundError:
    print("Error: The file 'CarOwners_translated.csv' was not found.")
    sys.exit(1)
except pd.errors.EmptyDataError:
    print("Error: The file is empty.")
    sys.exit(1)
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    sys.exit(1)

# Define a function to merge Province, City, and Address into a new column "Full Address"
def merge_address_columns(df):
    """
    Merges 'Province', 'City', and 'Address' columns into a new 'Full Address' column.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing the address information.
    
    Returns:
    pd.DataFrame: The DataFrame with the new 'Full Address' column.
    """
    df['Full Address'] = df['Province'].fillna('').astype(str) + ', ' + df['City'].fillna('').astype(str) + ', ' + df['address'].fillna('').astype(str) + ', ' + df['post code'].fillna('').astype(str)
    df['Full Address'] = df['Full Address'].str.replace(r'(^, |, $)', '', regex=True)  # Clean up any leading/trailing commas

    # Store the columns to be dropped in a separate DataFrame
    garbage_df = df[['Province', 'City', 'address', 'post code', 'Monthly salary', 'marriage', 'educate', 'color', 'gender', 'Birthday', 'industry', 'Unnamed: 21']].copy()

    # Drop the original columns
    df.drop(columns=['Province', 'City', 'address', 'post code', 'Monthly salary', 'marriage', 'educate', 'color', 'gender', 'Birthday', 'industry', 'Unnamed: 21'], inplace=True)
  
    return df, garbage_df

# Merge the address columns and get the garbage DataFrame
df, garbage_df = merge_address_columns(df)

# Rename columns
df.rename(columns={'Mail': 'Email', 'Engine No.': 'Engine Number', 'ID card': 'ID Number'}, inplace=True)

def save_dataframe_to_csv(df, file_path):
    """
    Saves a pandas DataFrame to a CSV file at the specified file path.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to save.
    file_path (str): The file path where the CSV file will be saved.
    """
    try:
        sys.setrecursionlimit(10000)  # Increase recursion limit if necessary (use caution)
        
        if df.empty:
            print("Warning: The DataFrame is empty. No file will be saved.")
        else:
            df.to_csv(file_path, index=False)
            print(f"File saved successfully to {file_path}")
    except RecursionError as rec_err:
        print(f"Recursion error encountered: {rec_err}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Usage example
cleaned_file_path = r"C:\Users\PROTEXXA\Desktop\NationWideChina\CleanedCarOwners\CleaningCarOwners.csv"  # Use raw string for file paths
garbage_file_path = r"C:\Users\PROTEXXA\Desktop\NationWideChina\CleanedCarOwners\Garbage\DroppedColumns.csv"  # Path for garbage file

# Save DataFrame to the file
save_dataframe_to_csv(df, cleaned_file_path)

# Save the garbage DataFrame to a separate file
save_dataframe_to_csv(garbage_df, garbage_file_path)


In [ ]:
import pandas as pd
import re
import warnings 

# Suppress FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)
 
# Function to validate email format (must be alphanumeric before the @)
def is_valid_email(email):
    if not isinstance(email, str):
        return False 
    email_regex = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    return re.match(email_regex, email) is not None and bool(re.search(r'[a-zA-Z0-9]', email.split('@')[0]))

# Function to process the CSV file
def process_csv(input_file, garbage_file, cleaned_file):
    # Read the CSV file into a DataFrame with low_memory=False
    df = pd.read_csv(input_file, low_memory=False)

    # Create a garbage DataFrame for invalid/duplicate rows
    garbage_df = pd.DataFrame(columns=df.columns)

    # Check for invalid emails (non-alphanumeric characters in the local part or incorrect format)
    invalid_email_rows = df[~df['Email'].apply(is_valid_email)]

    # Append invalid email rows to the garbage DataFrame
    if not invalid_email_rows.empty:
        garbage_df = pd.concat([garbage_df, invalid_email_rows])

    # Drop invalid email rows from the original DataFrame
    df = df.drop(invalid_email_rows.index)

    # Check for duplicate Emails, ID Numbers, Frame numbers,
    duplicates = df[df.duplicated(subset=['Email', 'ID Number', 'Frame number',], keep = 'first')]

    # Append duplicate rows to the garbage DataFrame
    if not duplicates.empty:
        garbage_df = pd.concat([garbage_df, duplicates])

    # Drop duplicate rows from the original DataFrame
    df = df.drop(duplicates.index)

   # Replace specific emails with 'Null'
    df['Email'] = df['Email'].replace({
        'noemail@email.com': 'Null',
        'nomail@mail.com': 'Null',
        'noema@email.com': 'Null'
    })

    # Save the garbage DataFrame to a CSV file
    garbage_df.to_csv(garbage_file, index=False)

    print(f"Invalid and duplicate rows have been saved to {garbage_file}")
     
    # Save the cleaned data (valid rows) to a new CSV file
    df.to_csv(cleaned_file, index=False)
    print(f"Cleaned data has been saved to {cleaned_file}")

# Example usage
input_file = r"C:\Users\PROTEXXA\Desktop\NationWideChina\CleanedCarOwners\CleaningCarOwners.csv"# Path to your input CSV file
garbage_file = r"C:\Users\PROTEXXA\Desktop\NationWideChina\CleanedCarOwners\Garbage\InvalidRows2.csv"   # Path where you want to save the invalid/duplicate rows
cleaned_file = r"C:\Users\PROTEXXA\Desktop\NationWideChina\CleanedCarOwners\CleanedCarOwner2.csv"  # Path where you want to save the cleaned data

process_csv(input_file, garbage_file, cleaned_file)